<div align="center">
  <img src="https://raw.githubusercontent.com/FabioMijango/Zombolab/refs/heads/main/Zombolab-Logo.png" alt="Zombolab Logo" width="250">
</div>

<h1>Zombolab</h1>

<p>
  <strong>Run a Project Zomboid Dedicated Server using Google Colab and Google Drive.</strong>
</p>
<br>

---

## 🧟 What is Zombolab?

**Zombolab** allows you to host a **Project Zomboid Dedicated Server** for free. It uses **LocalToNet** as a tunnel to expose an IP address and connect to the server, while saving your world data persistently in **Google Drive**.

Inspired by [Minecolab project](https://github.com/thecoder-001/MineColab) and [MineColab_Improved](https://github.com/N-aksif-N/MineColab_Improved).

---

## 🌐 1. Prerequisite: Tunnel Setup (Localtonet)

Before running the code, you need to configure the UDP connection.

1.  Register at [localtonet.com](https://localtonet.com/).
2.  Go to **TCP - UDP Tunnels**.
3.  Create a tunnel with these **settings**:
    * **Protocol:** `UDP`
    * **Local Address:** Leave as default (`127.0.0.1`)
    * **Local Port:** `16261`
    * **Server**: Select the server closest to you.
    * **Auth Token**: You can use the `Default Token`.
4.  Click **Create**.

---

## 🚀 2. How to Run

1.  **Set up:** Run the `Set up` cell. It will install everything it needs to run the server. Wait for the installation to finish.
    > *If an error occurs during installation, try running the cell again.*

2.  **Put Token:** Once the setup is finished, scroll down to the configuration block and paste your Token in the `TOKEN = "YOUR_TOKEN"` variable.
    > You can find your **AUTH TOKEN** in the `Dashboard` section at [localtonet.com](https://localtonet.com/).

3.  **Run Server:** Execute the `Run server` cell.

4.  **Activate Tunnel:** Watch the logs. When you see a message like this:
    ```
    ⏳ Waiting for tunnel connection...
    ```
    Go to [**TCP - UDP Tunnels**](https://localtonet.com/tunnel/tcpudp) on the website and click the **Start** button on your tunnel to bring it Online.

5.  **Success:** If everything connects correctly, you will see something like this in the output:
    ```text
    ✅ TUNNEL SUCCESSFULLY CONNECTED!
    👇 YOUR IP TO ENTER THE GAME 👇
    --------------------------------------------------
    🚀 URL: someip.localto.net
    📦 PORT: somePort
    --------------------------------------------------
    ```
    > You can also copy the IP address directly from the **TCP - UDP Tunnels** section on the website.
> ⚠️ **Note:** The server takes about **5-10 minutes** to download and start. Wait for the `SERVER STARTED` message in the logs before trying to join.

-----

In [ ]:
# @markdown ##**[❗]  Set up** {display-mode: "form"}
Version = 'b41' # @param ["b41", "b42 MP - unstable"]

import os
from os.path import exists

print("\n--- 🛠️ PHASE 1: SYSTEM PREPARATION 🛠️ ---")
# 1. Architecture y Update
!sudo dpkg --add-architecture i386
!sudo apt update -y > /dev/null 2>&1 && echo "✅ Update and i386 ready." || echo "❌ The update failed.."
# 2. Required libs
!sudo apt install lib32gcc-s1 lib32stdc++6 -y > /dev/null 2>&1 && echo "✅ Libs installed." || echo "❌ Libs fail to install."


print("\n--- 🎮 PHASE 2: STEAMCMD INSTALLATION 🎮 ---")
# 3. Accept licenses automatically (Avoid blocking)
!echo steam steam/license note '' | debconf-set-selections
!echo steam steam/question select "I AGREE" | debconf-set-selections
# 4. Install SteamCMD
!sudo apt install steamcmd -y > /dev/null 2>&1 && echo "✅ SteamCMD installed." || echo "❌ SteamCMD installation failed."


print("\n--- ℹ️ PHASE 3: CONNECT GOOGLE DRIVE ℹ️ ---")
SERVER_PATH = "/content/pzserver"
SAVES_PATH = "/content/drive/MyDrive/ZomboidSaves"
# 5. Conect drive.
if not exists("/content/drive"):
  print("Connecting drive...")
  from google.colab import drive
  drive.mount('/content/drive')
  print("✅ Drive connected.")
else:
  print("✅ Drive alredy conected.")
# 6. Link Drive saves to Server
!mkdir -p "{SAVES_PATH}"
!rm -rf /root/Zomboid
!ln -s "{SAVES_PATH}" /root/Zomboid


print("\n--- 🧟 PHASE 4: DOWNLOAD PROJECT ZOMBOID 🧟 ---\n  This may take some minutes...")
# 7. Create dirs
!mkdir -p "{SERVER_PATH}"
!mkdir -p "{SAVES_PATH}"
# 8. Run the download
verToInstall = 'app_update 380870 validate'
if(Version == 'b42 MP - unstable'):
  verToInstall = 'app_update 380870 -beta unstable - unstable validate'

!/usr/games/steamcmd +force_install_dir "{SERVER_PATH}" +login anonymous +"{verToInstall}" +quit # > /dev/null 2>&1 && echo "✅ Project Zomboid downloaded successfully." || echo "❌ The game download failed... Try again"


print("\n--- 🌐 PHASE 5: LOCALTONET INSTALLATION 🌐 ---")
# 9. Install LocalToNet
!sudo curl -fsSL https://localtonet.com/install.sh | sh > /dev/null 2>&1 && echo "✅ Localtonet installed." || echo "❌ Localtonet installation failed."

print("\n--- 🏁 EVERYTHING READY 🏁 ---")



---



In [ ]:
# @title Run server
TOKEN = 'YOUR_TOKEN' # @param {type: 'string'}

import subprocess
import time
import sys
import os
import threading

if TOKEN == "YOUR_TOKEN" or TOKEN == "" or TOKEN == None:
  sys.exit("\n❌ ERROR: No token provided")

SERVER_EXEC = "/content/pzserver/start-server.sh"
if not os.path.exists(SERVER_EXEC):
    sys.exit("\n❌ ERROR: Missing files.\n☭ Please run the installation script first and verify that it has completed successfully..")
print("✅ Verification complete: System ready to boot.")

# 1. Open file to log
log_file = open("tunnel.log", "w")


# 2. The tunnel starts in the background.
try:
    tunnel_process = subprocess.Popen(
        ["localtonet", "--authtoken", TOKEN],
        stdout=log_file,
        stderr=log_file
    )
    print("✅ Tunneling process initiated (PID: {}).".format(tunnel_process.pid))
except FileNotFoundError:
    sys.exit("❌ ERROR: The program 'localtonet' cannot be found. Did you run the installation?")

# 3. Wait and verify connection by reading the log
print("⏳ Waiting for connection confirmation (10 seg)...")
print("-" * 50)


print("⏳ Waiting for tunnel connection...")
connected = False
while not connected:
    time.sleep(10)

    try:
        with open("tunnel.log", "r") as f:
            log_content = f.read()

            if log_content.strip():
                lines = log_content.strip().splitlines()
                last_line = lines[-1]

                print(f"\r State: {last_line[:90]}...", end="")

                if "ADDED" in log_content:
                    connected = True
                    print("\n\n✅ TUNNEL SUCCESSFULLY CONNECTED!")
                    print("👇 YOUR IP TO ENTER THE GAME 👇")
                    print("-" * 50)
                    for line in lines:
                        if "ADDED" in line:
                            parts = line.split(' ')
                            if len(parts) > 4:
                                url_port = parts[5].strip()
                                if ':' in url_port:
                                    url, port = url_port.split(':')
                                    print(f"🚀 URL: {url}")
                                    print(f"🛖 PORT: {port}")
                                else:
                                    print(f"🚀 {line}")
                            else:
                                print(f"🚀 {line}")
                    print("-" * 50)

    except FileNotFoundError:
      pass

# 2. Make sure link exists
!rm -rf /root/Zomboid
!ln -s "{SAVES_PATH}" /root/Zomboid

print("\n--- 🧀 STEP 2: STARTING PROJECT ZOMBOID ---")
# Run the server initialization
!/content/pzserver/start-server.sh -adminpassword admin